In [1]:
import time
nb_start_time = time.time()

import pandas as pd
import numpy as np
from urllib.request import urlopen
from zipfile import ZipFile
from io import BytesIO

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
# working on Prince or locally?
%pwd

'/Users/bryant/Documents/nyuHpcTopicModeling'

In [3]:
# read in data
start_time = time.time()

file = 'mergedProjectsAbstracts.csv'
df = (pd.read_csv(file,skipinitialspace=True,encoding='utf-8',
                 dtype={'PROJECT_ID': object,
                        'PROJECT_TERMS': object,
                        'PROJECT_TITLE': object,
                        'DEPARTMENT': str,
                        'AGENCY': str,
                        'PROJECT_START_DATE': str,
                        'PROJECT_END_DATE': str,
                        'ORGANIZATION_CITY': str,
                        'CFDA_CODE': str,
                        'FY': int,
                        'FY_TOTAL_COST': float,
                        'FY_TOTAL_COST_SUB_PROJECTS': float                     
                       }))

print('\n')
elapsed_time = time.time() - start_time
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

/Users/bryant/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)




00:00:45


In [4]:
# drop if abstract missing
df_nomiss = df[df.ABSTRACT.notnull()]
print(df.shape)
print(df_nomiss.shape)

(1040239, 26)
(1032895, 26)


In [5]:
# subset for testing, otherwise leave commented
df_modeling = df_nomiss[:5000] # 1x

In [6]:
# define our vectorizer
my_vectorizer = CountVectorizer(max_df=0.10,
                                  min_df=0.005,
                                  ngram_range = (0,2))

In [7]:
start_time = time.time()

# set up
corpus = df_modeling.ABSTRACT

# vectorize
doc_term_matrix = my_vectorizer.fit_transform(corpus)
doc_term_features = my_vectorizer.get_feature_names()

# run LDA
LDA = LatentDirichletAllocation(n_topics=5, random_state=1)  
LDA.fit(doc_term_matrix)

# initialize list for topics
topicList = []
for i,topic in enumerate(LDA.components_):
    ithTopic = [doc_term_features[i] for i in topic.argsort()[-20:]]
    topicList.append(ithTopic)
topicListDf = pd.DataFrame(topicList)

# matrix where each row is an abstract, each column a topic. Each cell is value of that topic for that abstract.
topic_values = LDA.transform(doc_term_matrix)

# save each project's most relevant topic in a new variable, and its valence in a further variable
df_modeling['primeTopicId'] = topic_values.argmax(axis=1)
df_modeling['primeTopicValence'] = topic_values.max(axis=1)

elapsed_time = time.time() - start_time
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

/Users/bryant/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


00:00:32


/Users/bryant/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/bryant/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
df_modeling.head()

,Unnamed: 0,PROJECT_ID,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,...,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ABSTRACT,primeTopicId,primeTopicValence
0,0,1098983,Deposition; design; experimental study; Hydroc...,COMBUSTION CHAMBER DEPOSIT EFFECTS ON ENGINE H...,EPA,EPA,NaN,R824970C002,1/1/2000,6/30/1997,...,NaN,NaN,NaN,NaN,2000,496012.0,NaN,(1) To design a carefully-controlled experimen...,0,0.553078
1,1,1098991,Aerosols; Air Pollution; Atmosphere; Chemicals...,EXPERIMENTAL INVESTIGATION OF THE EVOLUTION OF...,EPA,EPA,NaN,R824970C010,1/1/2000,5/31/1998,...,NaN,NaN,NaN,NaN,2000,75000.0,NaN,The purpose of this research project is to con...,0,0.876265
2,2,1098989,Alcohols; Chemistry; Ethers; Experimental Mode...,FUNDAMENTAL STUDY ON HIGH TEMPERATURE CHEMISTR...,EPA,EPA,NaN,R824970C008,1/1/2000,12/31/1995,...,NaN,NaN,NaN,NaN,2000,265605.0,NaN,Experimental and modeling studies are performe...,4,0.707042
3,3,1098997,Air; Biological; Carbon; Chemicals; Gases; Liq...,INTEGRATING MODELS FOR PREDICTING POLLUTION TR...,EPA,EPA,NaN,R825370C005,1/1/2000,1/1/2000,...,NaN,NaN,NaN,NaN,2000,NaN,NaN,The objective of this project is to develop a ...,4,0.763460
4,4,1099005,Area; base; Emerging Technologies; Formulation...,ENVIRONMENTAL ASPECTS OF POLYMER FORMULATIONS,EPA,EPA,NaN,R825370C013,1/1/2000,1/1/2000,...,NaN,NaN,NaN,NaN,2000,NaN,NaN,An objective of this project is a survey and d...,4,0.983181


In [10]:
df_modeling.primeTopicId.value_counts()

3    1603
0    1196
4    1014
1     837
2     350
Name: primeTopicId, dtype: int64

In [11]:
# what does the top topic consist of?
topicList[3]

['stem',
 'university of',
 'educational',
 'computer',
 'researchers',
 'research and',
 'student',
 'public',
 'undergraduate',
 'engineering',
 'materials',
 'programs',
 'scientific',
 'activities',
 'training',
 'graduate',
 'social',
 'school',
 'center',
 'learning']